Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip -q install ultralytics
    !pip -q install paddleocr[all]
    if not os.path.exists('license-plate-finetune-v1s.pt'):
        !wget https://huggingface.co/morsetechlab/yolov11-license-plate-detection/resolve/main/license-plate-finetune-v1s.pt



In [ ]:
from ultralytics import YOLO
from pathlib import Path
from paddleocr import PaddleOCR

license_plate_detection_model = YOLO('license-plate-finetune-v1s.pt')

car_detection_model = YOLO("yolo11n.pt")
ocr_en = PaddleOCR(use_angle_cls=True, lang='en')
    


In [ ]:
def parse_ocr_result(ocr_result):
    """Parse PaddleOCR result and extract readable text with confidence scores."""
    if not ocr_result or len(ocr_result) == 0:
        return []
    
    parsed_results = []
    for detection in ocr_result:
        if detection and len(detection) > 0:
            rec_texts = detection.get('rec_texts', [])
            rec_scores = detection.get('rec_scores', [])
            
            for text, score in zip(rec_texts, rec_scores):
                parsed_results.append({
                    'text': text,
                    'confidence': round(score, 3)
                })
    
    return parsed_results

In [ ]:
def detect_cars_and_plates(image_path: Path | str):
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path, classes=[2,5,7])  # COCO classes for car, bus, truck
    output_plates = []
    for car in car_results[0].boxes:
        # Extract car bounding box
        x1, y1, x2, y2 = map(int, car.xyxy[0])
        car_image = car_results[0].orig_img[y1:y2, x1:x2]
        
        plate_results = license_plate_detection_model.predict(source=car_image)
        
        for plate in plate_results[0].boxes:
            px1, py1, px2, py2 = map(int, plate.xyxy[0])
            license_plate_image = car_image[py1:py2, px1:px2]
            ocr_result = ocr_en.predict(license_plate_image)
            
            # Parse the OCR result to get readable text
            parsed = parse_ocr_result(ocr_result)
            output_plates.append(parsed)
            
    return output_plates

In [ ]:


def display_plate_text(parsed_results):
    """Display the license plate text in a readable format."""
    if not parsed_results:
        return "No text detected"
    
    # Combine all text segments
    full_text = ' '.join([r['text'] for r in parsed_results])
    avg_confidence = sum([r['confidence'] for r in parsed_results]) / len(parsed_results)
    
    print(f"  License Plate: {full_text}")
    print(f"  Confidence: {avg_confidence:.1%}")
    print(f"  Details: {parsed_results}")
    
    return full_text

In [ ]:
if 'google.colab' in sys.modules:
    if "bmwalleng.png" not in os.listdir("."):
      !wget https://github.com/AFAskar/dlimageclass/blob/main/bmwalleng.png?raw=true
      !mv bmwalleng.png?raw=true bmwalleng.png

# list of data_plate
data_plates =[]
data_plate ={
  "img",
  "plate_data"
}
images = ['./bmwalleng.png']
for img_path in images:

    print(f"\nProcessing image: {img_path}")
    plates = detect_cars_and_plates(img_path)
    
    if plates:
        print(f"\nFound {len(plates)} license plate(s):")
        for i, plate_data in enumerate(plates, 1):
            print(f"\nPlate #{i}:")
            data_plate = {
              "img": img_path,
              "plate_data": plate_data
            }
            data_plates.append(data_plate)
            display_plate_text(plate_data)
            
    else:
        print("No license plates detected")
        
    



In [ ]:
from typing import Any
# draw bounding boxes and plate texts on the image
def visualize_plates(data_plates:list[dict[str,Any]]):
    import cv2
    import matplotlib.pyplot as plt

    for data_plate in data_plates:
        img_path = data_plate['img']
        plate_data = data_plate['plate_data']
        
        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Draw bounding boxes and texts
        for plate in plate_data:
            box = plate['box']  # Assuming 'box' key contains bounding box coordinates
            text = plate['text']
            confidence = plate['confidence']
            
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, f"{text} ({confidence:.1%})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        # Display the image
        plt.figure(figsize=(10, 6))
        plt.imshow(image)
        plt.axis('off')
        plt.show()
